In [1]:
%load_ext autoreload
%autoreload 2

# Exercise 2

<img src="./images/02.png" width=800>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.autonotebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from utils import train_network, View, set_seed
import mlflow
from torchinfo import summary
import os
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_score

/tmp/ipykernel_33109/4002559147.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns07_2'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [4]:
mlflow.set_experiment('Exercise07_2')

<Experiment: artifact_location='/kaggle/working/mlruns07_2/867018801752321722', creation_time=1750935992984, experiment_id='867018801752321722', last_update_time=1750935992984, lifecycle_stage='active', name='Exercise07_2', tags={}>

In [5]:
torch.backends.cudnn.deterministic = True
set_seed(42)

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Dataset and DataLoader

In [7]:
class AutoencodDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        # x, y = self.dataset.__getitem__(index)
        x, y = self.dataset[index]
        return  x, x

In [8]:
train_data = AutoencodDataset(torchvision.datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = torchvision.datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoencodDataset(test_data_xy)
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=batch_size)

## Model

In [9]:
D = 28 * 28
n = 64
C =1
classes = 10

In [10]:
def getLayer(in_size, out_size):
    return nn.Sequential(
        nn.Linear(in_size, out_size),
        nn.BatchNorm1d(out_size),
        nn.ReLU()
    )

In [12]:
auto_encoder = nn.Sequential(
    nn.Flatten(),
    getLayer(D, D//2),
    getLayer(D//2, D//3),
    getLayer(D//3, D//4),
    nn.Linear(D//4, n),
)

auto_decoder = nn.Sequential(
    getLayer(n, D//4),
    getLayer(D//4, D//3),
    getLayer(D//3, D//2),
    nn.Linear(D//2, D),
    nn.Sigmoid(),
    View(-1, 1, 28, 28)
)

auto_encod_decode = nn.Sequential(
    auto_encoder,
    auto_decoder,
)

## Training

In [13]:
loss_func = nn.MSELoss()
epochs = 50
optimizer = optim.AdamW(auto_encod_decode.parameters())
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size,
    'optimizer': optimizer.defaults
    }

In [ ]:
with open('model_summary.txt', 'w') as f:
    f.write(str(summary(auto_encod_decode, inpt_size=(batch_size, C, 28, 28))))
with mlflow.start_run(nested=True, run_name='exercise_2'):
    mlflow.log_artifact('model_summary.txt')
    mlflow.log_params(params)

    results = train_network(
        model=auto_encod_decode,
        optimizer=optimizer,
        loss_func=loss_func,
        train_loader=train_loader,
        valid_loader=test_loader,
        epochs=epochs,
        device=device,
        checkpoint_file_save='model.pth',
        
    )

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

#

## Results

In [14]:
def load_model_from_mlflow(
    run_id, artifact_path, model, device
    ):
    artifact_uri = f'runs:/{run_id}/{artifact_path}'
    checkpoint_path = mlflow.artifacts.download_artifacts(artifact_uri=artifact_uri)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer_state_dict = checkpoint['optimizer_state_dict']
    results = checkpoint['results']
    epoch = checkpoint['epoch']
    model.eval()
    model.to(device)
    return model, results, epoch

In [15]:
run_id = 'f6dca303a87242c3be031d1a96009107'
artifact_path = 'model.pth'
model, *_ = load_model_from_mlflow(
        run_id=run_id,
        artifact_path=artifact_path,
        model=auto_encod_decode, 
        device=device
    )

/tmp/ipykernel_33109/2809272947.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


In [16]:
test_loader_xy = DataLoader(test_data_xy, batch_size=batch_size)

In [24]:
original_images = []
encoded_representations = []
labels = []
model = model.to('cpu')
model = model.eval()
with torch.no_grad():
    for image, label in tqdm(test_loader_xy):
        images = image.view(image.size(0), -1).to(device)
        original_images.append(images)
        encoded = auto_encoder(images)
        encoded_representations.append(encoded.cpu().numpy())
        labels.append(label.cpu().numpy())

original_represenations = np.vstack(original_images)
encoded_representations = np.vstack(encoded_representations)
labels = np.hstack(labels)

print(f"Running K-Means on original images (shape: {original_represenations.shape})")
kmeans_original = KMeans(n_clusters=classes, random_state=42, n_init=10)
kmeans_original.fit(original_represenations)
original_predictions = kmeans_original.labels_

# Evaluate Homogeneity Score for original images
homogeneity_original = homogeneity_score(labels, original_predictions)
print(f"Homogeneity Score (Original Images): {homogeneity_original:.4f}")

print(f"Running K-Means on encoded images (shape: {encoded_representations.shape})")
kmeans_original = KMeans(n_clusters=classes, random_state=42, n_init=10)
kmeans_original.fit(encoded_representations)
encoded_predictions = kmeans_original.labels_

# Evaluate Homogeneity Score for original images
homogeneity_original = homogeneity_score(labels, encoded_predictions)
print(f"Homogeneity Score (Encoded Images): {homogeneity_original:.4f}")

  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:02<00:00, 36.84it/s]


Running K-Means on original images (shape: (10000, 784))
Homogeneity Score (Original Images): 0.5002
Running K-Means on encoded images (shape: (10000, 64))
Homogeneity Score (Encoded Images): 0.5195
